In [ ]:
# edit this variable to set de data streams

datasets = [
    'sine_rw_10_2341'
]

In [1]:
import pandas as pd
import pickle
import numpy as np
import statsmodels.stats.api as sms
import scipy
import os

In [2]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [3]:
models = ["gim_ilstm_50hs", "single_ilstm_50hs", "multiple_ilstm_50hs"]

alpha = 0.05

In [4]:
models_dict = {
    "cpnn_clstm_50hs": "cPNN",
    "single_clstm_50hs": "cLSTM",
    "multiple_clstm_50hs": "cLSTMs",
    "ensemble_clstm_50hs_5ws": "ensemble"
}

In [5]:
tasks_dict = {
    "1": "SINE1+",
    "2": "SINE2+",
    "3": "SINE1-",
    "4": "SINE2-"
}

In [6]:
def compute_performance(dataset, models, metric="accuracy"):
    perf_dict = {}
    perf_interval = {}
    for model in models:
        try:
            perf_dict[model] = {}
            with open(f"performance/{dataset}/{model}/test_then_train.pkl", "rb") as f:
                perf = pickle.load(f)
            for k in perf:
                perf[k] = np.asarray(perf[k])
            for k in [metric]:
                perf_dict[model][k] = {
                    "[1,50]": np.mean(perf[k][:,:,:50], axis=2),
                    "[1,100]": np.mean(perf[k][:,:,:100], axis=2),
                    "(100,)": np.mean(perf[k][:,:,100:], axis=2),
                    "[1,)": np.mean(perf[k], axis=2)
                }
        except Exception as e:
            print(e)
            del perf_dict[model]
            print(model, "not present")
    
    return perf_dict

In [7]:
def calculate_confint(perf):
    conf = sms.DescrStatsW(perf).tconfint_mean()
    return np.round(conf, 3), (conf[0]+conf[1])/2

In [8]:
def check_normality(data):
    pvalue = np.round(scipy.stats.shapiro(data).pvalue, 3)
    if pvalue>alpha:
        return True
    return False

In [9]:
def normality_test(dataset, models, metric="accuracy"):
    perf = compute_performance(dataset, models)
    perf_batches = {}
    tasks = dataset.split("_")[-1]

    for model in perf:
        perf_batches[model] = []
        for t in range(0,perf[model][metric][list(perf[model][metric].keys())[0]].shape[1]):
            d = {}
            for b in perf[model][metric]:
                d[b] = perf[model][metric][b][:,t]
            perf_batches[model].append(d)
            
    test_df = pd.DataFrame(
        columns=(
            ["task", "batches"] +
            [models_dict[m] for m in models]
        )
    )
    
    for t in range(0,len(perf_batches[models[0]])):
        for b in perf_batches[models[0]][t]:
            row = {
                "task": tasks_dict[tasks[t]],
                "batches": b,
            }
            for model in models:
                row[models_dict[model]] = check_normality(perf_batches[model][t][b])
            test_df = test_df.append(row, ignore_index=True)
    return test_df

In [10]:
import warnings
warnings.filterwarnings('ignore')

for d in datasets:
    print(d)
    test_df = normality_test(d, models)

    path = f"performance/{d}/_prob_test"
    if not os.path.isdir(path):
        os.makedirs(path)
    test_df.to_csv(os.path.join(path, f"normality_{d}.csv"), index=False)
    test_df.to_excel(os.path.join(path, f"normality_{d}.xlsx"), index=False)

sine_rw_10_13
sine_rw_10_24
sine_rw_10_31
sine_rw_10_42


In [11]:
test_df

,task,batches,iPNN,iLSTM,iLSTMs
0,SINE2-,"[1,50]",True,True,False
1,SINE2-,"[1,100]",True,False,True
2,SINE2-,"(100,)",True,True,True
3,SINE2-,"[1,)",True,True,True
4,SINE2+,"[1,50]",True,True,True
5,SINE2+,"[1,100]",True,True,True
6,SINE2+,"(100,)",True,True,True
7,SINE2+,"[1,)",True,True,True
